# Task Description: Filtering Metallica Albums by Year

## Overview

The goal of this project is to use prompt engineering and chain of thought (CoT) techniques to instruct a Language Model (LLM) to filter and display all Metallica albums released after a specified year. The data for this task will be retrieved from the Spotify.t:



In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from dotenv import load_dotenv
import requests
import openai 
import shutil
import os

os.environ['OPENAI_API_KEY'] = 'key'
CHROMA_PATH = "chroma_music"

Fetch data from Spotify API

In [2]:
url = "https://spotify23.p.rapidapi.com/search/"

querystring = {"q":"metallica", "type": "album", "limit": "50"}

headers = {
	"x-rapidapi-key": "31889f1ddfmsh96d42e48a3f8968p1484b5jsn5f8010587ad9",
	"x-rapidapi-host": "spotify23.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

# Extract albums from the JSON response
results = response.json().get('albums', {}).get('items', [])
print(type(results))
print(len(results))

<class 'list'>
50


Get needed data about album title and year

In [3]:
all_albums = []
titles = set()  # unique album titles

for result in results:
    release_year = result['data'].get('date', {}).get('year', '')
    album_title = result['data'].get('name', '')
    
    if release_year and album_title:
        if album_title not in titles:
            album = {
                'artist': 'Metallica',
                'album': album_title,
                'year': release_year
            }
            all_albums.append(album)
            titles.add(album_title)

In [4]:
# Create documents from the album data
documents = [Document(page_content=str(album), metadata=album) for album in all_albums]

# Split the documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)
chunks = text_splitter.split_documents(documents)

print(f'Split {len(all_albums)} documents into {len(chunks)} chunks.')

Split 37 documents into 37 chunks.


In [5]:
document = chunks[15]
print(document.page_content)
print(document.metadata)

{'artist': 'Metallica', 'album': 'The Metallica Blacklist', 'year': 2021}
{'artist': 'Metallica', 'album': 'The Metallica Blacklist', 'year': 2021, 'start_index': 0}


In [6]:
# Create a new DB from the documents.
embedding_function = OpenAIEmbeddings()
db = Chroma.from_documents(
    chunks, embedding_function, persist_directory=CHROMA_PATH
)

print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")   

Saved 37 chunks to chroma_music.


In [7]:
# Prepare the DB.
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [8]:
year_filter = 2020
query_text = f'List Metallica albums released after {year_filter}, focusing on unique and different albums.'

In [9]:
# Perform the query with a higher number of results
results = db.similarity_search_with_relevance_scores(query_text, k=20)  # Increased k to fetch more results

In [10]:
if len(results) == 0 or results[0][1] < 0.7:
    print(f'Unable to find matching results.') 

In [11]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---


Answer the question based on the above context: {question}.

"""

# Chain of Thought (CoT) steps
cot_steps = [
    'First, we need to find all "albums" where the integer value of "year" is greater than year_filter. For instance, if year_filter = 1999, we need to choose all albums where the year is greater than year_filter. For example, for "album" == "Death Magnetic": "year" = 2008 and 2008 > year_filter. That is why it is a suitable album for us and we need to print its year and title. Next, for "album" == "The Metallica Blacklist": "year" = 2021 and 2021 > year_filter. It is also suitable information. We need to find all albums where "year" > year_filter and print those.',
    'For instance, for year_filter = 1999, we have such result: "It was found 15 albums: 2023: 72 Seasons, 2008: Death Magnetic, 2021: The Metallica Blacklist, 2013: Metallica Through The Never (Music from the Motion Picture), 2016: Hardwired…To Self-Destruct, 2023: metallica, 2023: Metallica, 2003: St. Anger, 2016: Hardwired…To Self-Destruct (Deluxe), 2020: S&M2, 2022: Lux Æterna, 2024: METALLICA, 2000: I Disappear, 2022: Metallica (Remix), 2020: Metallica Remixes."'   
]


# Combine prompt and CoT steps
prompt_with_cot = PROMPT_TEMPLATE + "\n" + "\n".join(cot_steps)

print(prompt_with_cot)


Answer the question based only on the following context:

{context}

---


Answer the question based on the above context: {question}.


First, we need to find all "albums" where the integer value of "year" is greater than year_filter. For instance, if year_filter = 1999, we need to choose all albums where the year is greater than year_filter. For example, for "album" == "Death Magnetic": "year" = 2008 and 2008 > year_filter. That is why it is a suitable album for us and we need to print its year and title. Next, for "album" == "The Metallica Blacklist": "year" = 2021 and 2021 > year_filter. It is also suitable information. We need to find all albums where "year" > year_filter and print those.
For instance, for year_filter = 1999, we have such result: "It was found 15 albums: 2023: 72 Seasons, 2008: Death Magnetic, 2021: The Metallica Blacklist, 2013: Metallica Through The Never (Music from the Motion Picture), 2016: Hardwired…To Self-Destruct, 2023: metallica, 2023: Metallica, 2003: 

In [12]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(prompt_with_cot)

prompt = prompt_template.format(context=context_text, question=query_text)

model = ChatOpenAI()
response_text = model.invoke(prompt)

formatted_response = f"Response: {response_text}"
print(formatted_response)


Response: content='Based on the context provided, the Metallica albums released after 2020 are:\n\n1. 2023: 72 Seasons\n2. 2021: The Metallica Blacklist\n3. 2023: metallica\n4. 2023: Metallica\n5. 2022: Lux Æterna\n6. 2024: METALLICA\n7. 2022: Metallica (Remix)' response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 867, 'total_tokens': 958}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7d655333-7ba7-454c-8aba-efbfee176a28-0' usage_metadata={'input_tokens': 867, 'output_tokens': 91, 'total_tokens': 958}


Checking by filtering all albums after year_filter

In [13]:
# Function to filter albums by year
def filter_albums_by_year(all_albums, year_filter):
    filtered_documents = [album for album in all_albums if int(album['year']) > year_filter]
    
    if not filtered_documents:
        print("No albums found after the specified year.")
    else:
        filtered_documents_total = len(filtered_documents)
        print(f'It was found {filtered_documents_total} albums:')
        for album in filtered_documents:
            print(f" - {album['year']}: {album['album']}.")
            

In [14]:
year_filter = 2020        
filter_albums_by_year(all_albums, year_filter)

It was found 7 albums:
 - 2023: 72 Seasons.
 - 2021: The Metallica Blacklist.
 - 2023: metallica.
 - 2023: Metallica.
 - 2022: Lux Æterna.
 - 2024: METALLICA.
 - 2022: Metallica (Remix).
